## 先生成核矩阵保存起来

In [139]:
from shogun_tools import *
from shogun import GMNPSVM, CSVFile

def combined_kernel(file_type, data_name, operate_type):
    if file_type == '4':
        X, y = loadFromMat(data_name)
    elif file_type == '5':
        X, y = loadFromLibsvm(data_name)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)  
    if type(X_train)==scipy.sparse.csr.csr_matrix and type(X_test)==scipy.sparse.csr.csr_matrix:
        X_train=X_train.todense()
        X_test=X_test.todense()
    X_train=X_train.T
    X_test=X_test.T
    y_train=y_train.reshape(y_train.size,).astype('float64')
    y_test=y_test.reshape(y_test.size,).astype('float64')

    kernel = CombinedKernel()
    feats_train = CombinedFeatures()
    feats_test = CombinedFeatures()
    subkfeats_train = RealFeatures(X_train)
    subkfeats_test = RealFeatures(X_test)
    for i in range(-10,11):
        subkernel = GaussianKernel(pow(2,i+1))
        feats_train.append_feature_obj(subkfeats_train)
        feats_test.append_feature_obj(subkfeats_test)
        kernel.append_kernel(subkernel)
    kernel.init(feats_train, feats_train)
    tmp_train_csv = NamedTemporaryFile(suffix=data_name+'_combined.csv')
    if operate_type=='save':        
        km_train=kernel.get_kernel_matrix()
        f=CSVFile(tmp_train_csv.name, "w")
        kernel.save(f)
    elif operate_type=='load':
        f=CSVFile(tmp_train_csv.name, "r")
        kernel.load(f)
        
    labels=MulticlassLabels(y_train)

    mkl = MKLMulticlass(C, kernel, labels)

    mkl.set_epsilon(epsilon)
    mkl.parallel.set_num_threads(num_threads)
    mkl.set_mkl_epsilon(mkl_epsilon)
    mkl.set_mkl_norm(mkl_norm)

    mkl.train()
    kernel.init(feats_train, feats_test)
    out =  mkl.apply().get_labels()
    print out.shape
    print out, y_test
    print sum(out==y_test)/float(len(out))


In [149]:
times=30
C=100 #[1, 10, 100, 1000]    
epsilon=1e-5
mkl_epsilon=0.001
test_size=0.2
width=8
num_threads=32
mkl_norm=1

import time
start = time.time()
combined_kernel('4', 'plant','load')
end=time.time()
print 'use time : ' + str(end-start)



(188,)
[ 0.  3.  0.  2.  2.  3.  3.  1.  2.  1.  1.  2.  2.  1.  1.  2.  1.  2.
  0.  0.  1.  2.  2.  3.  1.  3.  1.  3.  1.  1.  1.  1.  1.  0.  1.  1.
  2.  2.  2.  0.  2.  1.  1.  3.  1.  2.  3.  1.  2.  3.  2.  2.  2.  2.
  1.  1.  1.  3.  1.  2.  1.  2.  1.  2.  1.  1.  1.  1.  2.  2.  2.  1.
  2.  2.  1.  2.  0.  3.  2.  1.  1.  2.  3.  1.  2.  1.  3.  3.  3.  0.
  1.  0.  3.  2.  3.  1.  1.  1.  3.  1.  2.  1.  0.  2.  1.  3.  1.  2.
  1.  1.  2.  1.  2.  2.  3.  1.  2.  1.  2.  3.  2.  1.  2.  3.  2.  1.
  0.  1.  2.  0.  1.  2.  2.  2.  1.  3.  0.  1.  2.  1.  0.  2.  2.  2.
  3.  1.  1.  2.  1.  1.  1.  3.  2.  3.  1.  2.  1.  1.  1.  1.  2.  1.
  1.  1.  2.  1.  2.  1.  3.  1.  2.  2.  1.  0.  3.  3.  2.  2.  2.  2.
  1.  2.  1.  1.  2.  1.  1.  3.] [ 0.  3.  0.  3.  2.  3.  3.  1.  2.  1.  1.  3.  2.  1.  1.  2.  1.  2.
  1.  1.  1.  3.  2.  3.  1.  3.  1.  0.  1.  1.  1.  1.  1.  0.  3.  1.
  2.  2.  2.  3.  2.  1.  1.  3.  1.  2.  3.  1.  2.  3.  0.  2.  2.  2.
  0.  1.  

In [33]:
k2.get_kernel_matrix()[1,0]

8.2535074399873327

In [28]:
k2.get_kernel_matrix().T==k1.get_kernel_matrix()

array([[ True, False, False, ..., False, False, False],
       [False,  True, False, ..., False, False, False],
       [False, False,  True, ..., False, False, False],
       ..., 
       [False, False, False, ...,  True, False, False],
       [False, False, False, ..., False,  True, False],
       [False, False, False, ..., False, False,  True]], dtype=bool)

In [32]:
k1.get_kernel_matrix()[0,1]

8.1576132247494435